# MCT1/CTP1 statistical analysis
Purpose: Determine statistical metrics

#### Table of contents:
<ul>
    <li><a href="#import">Part 0: Import dependencies & custom functions</a></li>
    <li><a href="#rna">Part 1: Analyze RNA-seq data</a></li>
    <li><a href="#proteomics">Part 2: Analyze proteomics data</a></li>
    <li><a href="#gcms">Part 3: Analyze GC-MS metabolomics data</a></li>
    <li><a href="#lcms">Part 4: Analyze LC-MS metabolomics data</a></li>
    <li><a href="#session">Part 5: Print session info</a></li>
    
</ul>


<a id="import"><h3>Part 0: Import dependencies & custom functions</h3></a>

In [1]:
from __future__ import division
import os
import sys
import pandas as pd
import numpy as np
from numpy import mean, std
from math import sqrt
from scipy.stats import ttest_ind, sem, f
import scipy.stats as st
import xpressplot as xp
import gzip

from py_funcs import \
    mean_confidence_interval, \
    mean_confidence_interval_norm, \
    cohen_d, \
    confidence_overlap, \
    g_decompress, \
    read_table

__path__ = os.getcwd()
print(__path__)

C:\Users\jorda\Desktop\projects\Metaboverse-manuscript\mct1_analysis\notebooks


<a id="rna"><h3>Part 1: Analyze RNA-seq data</h3></a>

In [3]:
# RNA d values
rna_url = os.path.abspath(os.path.join("..", "..", "data", "sce_mct1_omics", "sce_mct1_03hr_counts_diffx.tsv"))
rna_de = pd.read_csv(
    rna_url,
    sep='\t',
    index_col=0)

mct1_rnaseq_tpm = read_table(
    url=os.path.join(
        __path__,
        "..",
        "data",
        "mct1_rnaseq_data",
        "sce_mct1_deduped_tpm_threshold25.tsv")
)

gtf = pd.read_csv(
    os.path.join(
        __path__,
        "..",
        "data",
        "analysis_lists",
        "Saccharomyces_cerevisiae.R64-1-1.103.gtf.gz"),
    sep='\t',
    comment='#',
    low_memory=False,
    header=None)

orig_name_label='gene_id "'
orig_name_location=0
new_name_label='gene_name "'
new_name_location=1
gtf_genes = gtf.loc[gtf[2] == 'gene']
gtf_genes['original'] = gtf[8].str.split(';').str[orig_name_location]
gtf_genes['new'] = gtf[8].str.split(';').str[new_name_location]
gtf_genes['original'] = gtf_genes['original'].map(lambda x: x.lstrip(str(orig_name_label)).rstrip('"').lstrip('"').rstrip(' '))
gtf_genes['new'] = gtf_genes['new'].map(lambda x: x.lstrip(str(new_name_label)).rstrip('"').rstrip(' '))
gtf_genes = gtf_genes[['original','new']].copy()

In [4]:
gene_dict = {}
for index, row in gtf_genes.iterrows():
    if row[1] == 'source "sgd':
        gene_dict[row[0]] = row[0]
    else:
        gene_dict[row[0]] = row[1]

rna_de_renamed = rna_de.copy()
rna_de_renamed['new'] = rna_de_renamed.index.to_series().map(gene_dict).fillna(
    rna_de_renamed.index.to_series())
rna_de_renamed = rna_de_renamed.set_index('new')
rna_de_renamed.index.name = None

mct1_rnaseq_tpm_renamed = mct1_rnaseq_tpm.copy()
mct1_rnaseq_tpm_renamed['new'] = mct1_rnaseq_tpm_renamed.index.to_series().map(
        gene_dict
    ).fillna(
        mct1_rnaseq_tpm_renamed.index.to_series())
mct1_rnaseq_tpm_renamed = mct1_rnaseq_tpm_renamed.set_index('new')
mct1_rnaseq_tpm_renamed.index.name = None


In [5]:
"""
14251X4    WT
14251X6    mct1del
14251X10   WT
14251X12   mct1del
14251X16   WT
14251X18   mct1del
14251X22   WT
14251X24   mct1del
"""

rna_mct1 = mct1_rnaseq_tpm_renamed[[
    '14251X6',
    '14251X12',
    '14251X18',
    '14251X24']]
rna_wt = mct1_rnaseq_tpm_renamed[[
    '14251X4',
    '14251X10',
    '14251X16',
    '14251X22']]

In [6]:
rna_list = [
    'GLK1',
    'PGI1',
    'PFK1', 'PFK2',
    'FBP1',
    'FBA1', 'TPI1',
    'TDH1', 'TDH2', 'TDH3',
    'PGK1',
    'GPM1',
    'ENO1', 'ENO2',
    'PYK2', 'CDC19',
    'PYC1', 'PYC2',
    'CIT1', 'CIT2', 'CIT3',
    'ACO1', 'ACO2',
    'IDH1', 'IDH2',
    'LPD1', 'KGD1', 'KGD2',
    'LSC1', 'LSC2',
    'SDH1', 'SDH2', 'SDH3', 'SDH4',
    'FUM1',
    'MDH1',
    'GDH1', 'GDH2', 'GLN1',
    'ALT1', 'ALT2',
    'DLD1', 'DLD2', 'DLD3',
    'MAE1',
    'AAT1',
    'CTP1', 'DIC1'
]


In [7]:
for x in rna_list:
    try:
        print(x)
        print('fold change: '+ str(np.log2(
            (sum(rna_mct1.loc[x].values))
            / (sum(rna_wt.loc[x].values))))

        )
        print('Cohen\'s d: '+ str(
            cohen_d(
                rna_mct1.loc[x].values,
                rna_wt.loc[x].values)
            )
        )
        confidence_overlap(rna_mct1, rna_wt, x, confidence=0.95)
        print(
            rna_de_renamed.loc[x]
        )
        print()
        print()
    except:
        print('Skipping ' + x + '...')
        print()
        print()

for x in rna_list:
    try:
        fc = str(round(np.log2(
            (sum(rna_mct1.loc[x].values))
            / (sum(rna_wt.loc[x].values)))
            , 2))
        p = str("{:.2e}".format(rna_de_renamed.loc[x]['FDR']))
        d = str(round(
            cohen_d(
                rna_mct1.loc[x].values,
                rna_wt.loc[x].values)
            , 2))

        print(x, ' & 3 hr & ', fc, ' & ', p, ' & ', d, '\\\\ \hline')
    except:
        pass

GLK1
fold change: 0.006217076550739141
Cohen's d: 0.07879130797914291
Confidence intervals (0.95) OVERLAP
Overlap: 76.67400000000004 (95.17%)
    Interval 1: [413.239, 492.0948]
    Interval 2: [411.5278, 489.913]
log2FoldChange   -0.009015
FDR               0.981977
Name: GLK1, dtype: float64


PGI1
fold change: -0.125407015797946
Cohen's d: -1.7911083906714387
Confidence intervals (0.95) OVERLAP
Overlap: 25.19409999999999 (27.17%)
    Interval 1: [347.9485, 395.7919]
    Interval 2: [370.5978, 440.6856]
log2FoldChange   -0.141688
FDR               0.326110
Name: PGI1, dtype: float64


PFK1
fold change: -0.12781009543408475
Cohen's d: -0.8586058815921683
Confidence intervals (0.95) OVERLAP
Overlap: 32.31959999999998 (30.47%)
    Interval 1: [212.2324, 244.552]
    Interval 2: [196.5058, 302.5924]
log2FoldChange   -0.141277
FDR               0.481721
Name: PFK1, dtype: float64


PFK2
fold change: -0.09948072203711131
Cohen's d: -0.3546580181046955
Confidence intervals (0.95) OVERLAP
Ov

<a id="proteomics"><h3>Part 2: Analyze proteomics data</h3></a>

In [8]:
# Proteomics d values
raw_proteomics_url = os.path.abspath(os.path.join(__path__, "..", "data", "mct1_proteomics_data", "proteomics_values.txt"))

proteomics = pd.read_csv(
    raw_proteomics_url,
    sep='\t',
    index_col=0)
proteomics_mct1 = proteomics[[
    'mct1_1',
    'mct1_2',
    'mct1_3']]
proteomics_wt = proteomics[[
    'WT_1',
    'WT_2',
    'WT_3']]

In [9]:
protein_list = [
    'GLK1',
    'PGI1',
    'PFK1', 'PFK2',
    'FBP1',
    'FBA1', 'TPI1',
    'TDH1', 'TDH2', 'TDH3',
    'PGK1',
    'GPM1',
    'ENO1', 'ENO2',
    'PYK2', 'CDC19',
    'PYC1', 'PYC2',
    'CIT1', 'CIT2', 'CIT3',
    'ACO1', 'ACO2',
    'IDH1', 'IDH2',
    'LPD1', 'KGD1', 'KGD2',
    'LSC1', 'LSC2',
    'SDH1', 'SDH2', 'SDH3', 'SDH4',
    'FUM1',
    'MDH1',
    'GDH1', 'GDH2', 'GLN1',
    'ALT1', 'ALT2',
    'DLD1', 'DLD2', 'DLD3',
    'MAE1',
    'AAT1',
    'CTP1', 'DIC1'
]

In [10]:
for x in protein_list:
    print(x)
    print('fold change: '+ str(np.log2(
        (sum(proteomics_mct1.loc[x].values))
        / (sum(proteomics_wt.loc[x].values))))

    )
    print('Cohen\'s d: '+ str(
        cohen_d(
            proteomics_mct1.loc[x].values,
            proteomics_wt.loc[x].values)
        )
    )
    confidence_overlap(proteomics_mct1, proteomics_wt, x, confidence=0.95)
    print(
        ttest_ind(
            proteomics_mct1.loc[x].values,
            proteomics_wt.loc[x].values)
        )
    print()
    print()

for x in protein_list:
    try:
        fc = str(round(np.log2(
            (sum(proteomics_mct1.loc[x].values))
            / (sum(proteomics_wt.loc[x].values)))
            , 2))
        p = str("{:.2e}".format(
            ttest_ind(
                proteomics_mct1.loc[x].values,
                proteomics_wt.loc[x].values)[1]
            ))
        d = str(round(
            cohen_d(
                proteomics_mct1.loc[x].values,
                proteomics_wt.loc[x].values)
            , 2))

        print(x.capitalize(), '(' + x + ') & 12 hr & ', fc, ' & ', p, ' & ', d, '\\\\ \hline')
    except:
        pass

GLK1
fold change: 0.08065134061829879
Cohen's d: 0.3350083204279233
Confidence intervals (0.95) OVERLAP
Overlap: 1.6143779999999985 (14.65%)
    Interval 1: [8.956822, 10.5712]
    Interval 2: [3.725025, 14.74128]
Ttest_indResult(statistic=0.41029972231760914, pvalue=0.7026113976636237)


PGI1
fold change: 0.30322172510266227
Cohen's d: 1.755316712917468
Confidence intervals (0.95) OVERLAP
Overlap: 0.8013300000000001 (10.11%)
    Interval 1: [10.09367, 10.895]
    Interval 2: [4.543848, 12.46622]
Ttest_indResult(statistic=2.1498151418136113, pvalue=0.0980019275956061)


PFK1
fold change: 0.1752032728319478
Cohen's d: 2.0163661602849854
Confidence intervals (0.95) OVERLAP
Overlap: 1.5164089999999995 (41.06%)
    Interval 1: [7.895111, 11.13636]
    Interval 2: [7.44356, 9.41152]
Ttest_indResult(statistic=2.469534113656587, pvalue=0.06898124707650109)


PFK2
fold change: 0.26438973535772914
Cohen's d: 5.206150893952391
Confidence intervals (0.95) DO NOT overlap
Ttest_indResult(statistic=

<a id="gcms"><h3>Part 3: Analyze GC-MS metabolomics data</h3></a>

In [11]:
raw_metabolomics_url = os.path.abspath("../data/mct1_metabolomics_data/mct1_metabolomics_allValues.txt")
raw_metabolomics_url = os.path.abspath("..//data/mct1_metabolomics_data/mct1_metabolomics_allValues.txt")
metabolomics_180min = pd.read_csv(
    raw_metabolomics_url,
    sep='\t',
    index_col=0)

metabolomics_mct1_180min = metabolomics_180min[[
    'mct1KO at 3hr minR+Leu 1',
    'mct1KO at 3hr minR+Leu 2',
    'mct1KO at 3hr minR+Leu 3',
    'mct1KO at 3hr minR+Leu 4',
    'mct1KO at 3hr minR+Leu 5',
    'mct1KO at 3hr minR+Leu 6'
]]
metabolomics_wt_180min = metabolomics_180min[[
    'WT at 3hr minR+Leu 1',
    'WT at 3hr minR+Leu 2',
    'WT at 3hr minR+Leu 3',
    'WT at 3hr minR+Leu 4',
    'WT at 3hr minR+Leu 5'
]]

In [12]:
metabolomics_wt_180min.index.tolist()

[' 1,6-anhydro-Glucose ',
 ' 1-Monomyristin',
 ' 1-Octadecene ',
 ' 1-Palmitoylglycerol',
 ' 1-Steroylglycerol ',
 ' 2-Hydroxybutyric acid ',
 ' 2-Hydroxyglutaric acid',
 ' 2-Ketoisocaproic acid',
 ' 2-Monopalmitoylglycerol',
 ' 2-Steroylglycerol',
 ' 3-Phosphoglyceric acid ',
 ' 4-Hydroxyproline',
 ' Adenine ',
 ' Adenosine ',
 ' Adenosine monophosphate',
 ' Adipic acid ',
 ' Allantoin',
 ' alpha-D-Glucosamine 1-phosphate ',
 ' Arachidic acid ',
 ' Arachidonic Acid',
 ' Azelaic acid ',
 ' Benzoic acid ',
 ' Beta-Alanine ',
 ' Beta-Glycerophosphoric acid ',
 ' Beta-Hydroxyisovaleric acid',
 ' Capric acid ',
 ' Cellobiose',
 ' cis-Aconitic acid',
 ' Citric acid ',
 ' Cystathionine ',
 ' D-Fructose',
 ' D-Glucose',
 ' D-Malic acid ',
 ' D-Mannose',
 ' D-Ribose-5-phosphate',
 ' D-Threitol ',
 ' Eicosane ',
 ' Elaidic acid ',
 ' Ergosterol ',
 ' Ethanolamine',
 ' fructose-1,6-diphosphate',
 ' Fructose-6-phosphate ',
 ' Fumaric acid ',
 ' Galactonic acid',
 ' Glucose 6-phosphate',
 ' Glutam

In [13]:
gc_metabolomics = [
    ' Glucose 6-phosphate',
    ' Fructose-6-phosphate ',
    ' fructose-1,6-diphosphate',
    ' 3-Phosphoglyceric acid ',
    ' Phosphoenolpyruvate ',
    ' Pyruvic acid ',
    ' Glyoxylic acid ',
    ' Citric acid ',
    ' Isocitric acid ',
    ' Succinic acid ',
    ' Fumaric acid ',
    ' D-Malic acid ',
    ' L-Alanine',
    ' L-Lactic acid ',
    ' L-Aspartic acid',
    ' L-Glutamic acid',
    ' 2-Hydroxyglutaric acid',
]

In [14]:
for x in gc_metabolomics:
    print(x)
    print('fold change: '+ str(np.log2(
        (sum(metabolomics_mct1_180min.loc[x].values))
        / (sum(metabolomics_wt_180min.loc[x].values))))

    )
    print('Cohen\'s d: '+ str(
        cohen_d(
            metabolomics_mct1_180min.loc[x].values,
            metabolomics_wt_180min.loc[x].values)
        )
    )
    confidence_overlap(
        metabolomics_mct1_180min,
        metabolomics_wt_180min,
        x,
        confidence=0.95)
    print(
        ttest_ind(
            metabolomics_mct1_180min.loc[x].values,
            metabolomics_wt_180min.loc[x].values)
        )
    print()
    print()

 Glucose 6-phosphate
fold change: -0.20011907930316603
Cohen's d: -0.7455454296236933
Confidence intervals (0.95) OVERLAP
Overlap: 244797.8 (51.03%)
    Interval 1: [159487.5, 461161.5]
    Interval 2: [216363.7, 639233.1]
Ttest_indResult(statistic=-1.242876405429128, pvalue=0.24531776877675918)


 Fructose-6-phosphate 
fold change: 0.07352294272141563
Cohen's d: -0.24773711188150752
Confidence intervals (0.95) OVERLAP
Overlap: 1158888.9 (76.38%)
    Interval 1: [448194.1, 1607083.0]
    Interval 2: [413282.5, 1930510.0]
Ttest_indResult(statistic=-0.411436707751002, pvalue=0.690377411918478)


 fructose-1,6-diphosphate
fold change: 1.2029904090879533
Cohen's d: 0.7537557818439704
Confidence intervals (0.95) OVERLAP
Overlap: 322852.8 (50.62%)
    Interval 1: [128021.5, 529817.2]
    Interval 2: [-107976.7, 450874.3]
Ttest_indResult(statistic=1.2560256134041172, pvalue=0.2407314553354735)


 3-Phosphoglyceric acid 
fold change: -0.8582499979616038
Cohen's d: -0.9898843839328134
Confidenc

In [15]:
for x in gc_metabolomics:
    try:
        fc = str(round(np.log2(
            (sum(metabolomics_mct1_180min.loc[x].values))
            / (sum(metabolomics_wt_180min.loc[x].values)))
            , 2))
        p = str("{:.2e}".format(
            ttest_ind(
                metabolomics_mct1_180min.loc[x].values,
                metabolomics_wt_180min.loc[x].values)[1]
            ))
        d = str(round(
            cohen_d(
                metabolomics_mct1_180min.loc[x].values,
                metabolomics_wt_180min.loc[x].values)
            , 2))

        print(x[1:-1].capitalize(), '() & 3 hr & ', fc, ' & ', p, ' & ', d, '\\\\ \hline')
    except:
        pass

Glucose 6-phosphat () & 3 hr &  -0.2  &  2.45e-01  &  -0.75 \\ \hline
Fructose-6-phosphate () & 3 hr &  0.07  &  6.90e-01  &  -0.25 \\ \hline
Fructose-1,6-diphosphat () & 3 hr &  1.2  &  2.41e-01  &  0.75 \\ \hline
3-phosphoglyceric acid () & 3 hr &  -0.86  &  1.20e-01  &  -0.99 \\ \hline
Phosphoenolpyruvate () & 3 hr &  -0.8  &  3.46e-02  &  -1.51 \\ \hline
Pyruvic acid () & 3 hr &  -0.02  &  4.67e-01  &  -0.48 \\ \hline
Glyoxylic acid () & 3 hr &  0.79  &  4.67e-01  &  0.48 \\ \hline
Citric acid () & 3 hr &  -2.59  &  5.36e-04  &  -3.0 \\ \hline
Isocitric acid () & 3 hr &  0.77  &  3.25e-01  &  0.6 \\ \hline
Succinic acid () & 3 hr &  0.34  &  7.88e-01  &  0.16 \\ \hline
Fumaric acid () & 3 hr &  1.61  &  1.09e-04  &  4.15 \\ \hline
D-malic acid () & 3 hr &  1.94  &  7.74e-04  &  3.15 \\ \hline
L-alanin () & 3 hr &  4.34  &  7.85e-05  &  4.34 \\ \hline
L-lactic acid () & 3 hr &  0.55  &  5.40e-01  &  0.37 \\ \hline
L-aspartic aci () & 3 hr &  1.2  &  5.97e-02  &  1.34 \\ \hline
L-glu

<a id="lcms"><h3>Part 4: Analyze LC-MS metabolomics data</h3></a>

In [16]:
# CTP1 Metabolomics
raw_metabolomics_ctp1 = os.path.abspath(os.path.join("..", "data", "ctp1_metabolomics_data", "ctp1_metabolomics_allValues.txt"))

metabolomics = pd.read_csv(
    raw_metabolomics_ctp1,
    sep='\t',
    index_col=0)

ctp1_sr_wt_ev = metabolomics[[
    'WT_SR_EV1',
    'WT_SR_EV2',
    'WT_SR_EV3']]

ctp1_sr_wt_a1 = metabolomics[[
    'WT_SR_A1',
    'WT_SR_A2',
    'WT_SR_A3']]

ctp1_sr_mct_ev = metabolomics[[
    'MCT_SR_EV1',
    'MCT_SR_EV2',
    'MCT_SR_EV3']]

ctp1_sr_mct_a1 = metabolomics[[
    'MCT_SR_A1',
    'MCT_SR_A2',
    'MCT_SR_A3']]

In [20]:
lc_metabolomics = [
    'Glucose',
    'F6P',
    'F16BP',
    'Pyruvate',
    'CoA',
    'Citrate',
    'a-KG',
    'Succinate',
    'Fumarate',
    'Malate',
    'Glutamate',
    'Glutamine',
    'Aspartate',
    'Alanine',
    'Lysine'
]

In [23]:
print('mct1-del vs WT')
for x in lc_metabolomics:
    print(x)
    print('fold change: '+ str(np.log2(
        (sum(ctp1_sr_mct_ev.loc[x].values))
        / (sum(ctp1_sr_wt_ev.loc[x].values))))

    )
    print('Cohen\'s d: '+ str(
        cohen_d(
            ctp1_sr_mct_ev.loc[x].values,
            ctp1_sr_wt_ev.loc[x].values)
        )
    )
    confidence_overlap(
        ctp1_sr_mct_ev,
        ctp1_sr_wt_ev,
        x,
        confidence=0.95)
    print(
        ttest_ind(
            ctp1_sr_mct_ev.loc[x].values,
            ctp1_sr_wt_ev.loc[x].values)
        )
    print()
    print()

for x in lc_metabolomics:
    try:
        fc = str(round(np.log2(
            (sum(ctp1_sr_mct_ev.loc[x].values))
            / (sum(ctp1_sr_wt_ev.loc[x].values)))
            , 2))
        p = str("{:.2e}".format(
            ttest_ind(
                ctp1_sr_mct_ev.loc[x].values,
                ctp1_sr_wt_ev.loc[x].values)[1]
            ))
        d = str(round(
            cohen_d(
                ctp1_sr_mct_ev.loc[x].values,
                ctp1_sr_wt_ev.loc[x].values)
            , 2))

        print(x.capitalize(), '() & 12 hr & ', fc, ' & ', p, ' & ', d, '\\\\ \hline')
    except:
        pass


mct1-del vs WT
Glucose
fold change: 0.653243768268703
Cohen's d: 0.9121090954479899
Confidence intervals (0.95) OVERLAP
Overlap: 273631270.0 (66.23%)
    Interval 1: [39851030.0, 313482300.0]
    Interval 2: [-94258730.0, 318925400.0]
Ttest_indResult(statistic=1.117100936799547, pvalue=0.326518995251931)


F6P
fold change: -0.15374653177310058
Cohen's d: -0.526648097203928
Confidence intervals (0.95) OVERLAP
Overlap: 62516100.0 (31.72%)
    Interval 1: [39275870.0, 236390800.0]
    Interval 2: [122075300.0, 184591400.0]
Ttest_indResult(statistic=-0.64500955607865, pvalue=0.5540523010738307)


F16BP
fold change: -0.5338737287886945
Cohen's d: -1.9570466002307467
Confidence intervals (0.95) OVERLAP
Overlap: 63566500.0 (41.76%)
    Interval 1: [29046050.0, 168954000.0]
    Interval 2: [105387500.0, 181279200.0]
Ttest_indResult(statistic=-2.396882786706952, pvalue=0.0746082024667167)


Pyruvate
fold change: -0.06254146606871294
Cohen's d: -0.34719044545040956
Confidence intervals (0.95) OV

In [24]:
print('mct1-del+CTP1 vs mct1-del')
for x in lc_metabolomics:
    print(x)
    print('fold change: '+ str(np.log2(
        (sum(ctp1_sr_mct_a1.loc[x].values))
        / (sum(ctp1_sr_mct_ev.loc[x].values))))

    )
    print('Cohen\'s d: '+ str(
        cohen_d(
            ctp1_sr_mct_a1.loc[x].values,
            ctp1_sr_mct_ev.loc[x].values)
        )
    )
    confidence_overlap(
        ctp1_sr_mct_a1,
        ctp1_sr_mct_ev,
        x,
        confidence=0.95)
    print(
        ttest_ind(
            ctp1_sr_mct_a1.loc[x].values,
            ctp1_sr_mct_ev.loc[x].values)
        )
    print()
    print()


for x in lc_metabolomics:
    try:
        fc = str(round(np.log2(
            (sum(ctp1_sr_mct_a1.loc[x].values))
            / (sum(ctp1_sr_mct_ev.loc[x].values)))
            , 2))
        p = str("{:.2e}".format(
            ttest_ind(
                ctp1_sr_mct_a1.loc[x].values,
                ctp1_sr_mct_ev.loc[x].values)[1]
            ))
        d = str(round(
            cohen_d(
                ctp1_sr_mct_a1.loc[x].values,
                ctp1_sr_mct_ev.loc[x].values)
            , 2))

        print(x.capitalize(), '() & 12 hr & ', fc, ' & ', p, ' & ', d, '\\\\ \hline')
    except:
        pass

mct1-del+CTP1 vs mct1-del
Glucose
fold change: -0.11321061044799094
Cohen's d: -0.34050261230349893
Confidence intervals (0.95) OVERLAP
Overlap: 28684300.0 (10.48%)
    Interval 1: [148991200.0, 177675500.0]
    Interval 2: [39851030.0, 313482300.0]
Ttest_indResult(statistic=-0.41702882811414893, pvalue=0.6980663685111017)


F6P
fold change: 0.1537465317731004
Cohen's d: 0.5344165696248049
Confidence intervals (0.95) OVERLAP
Overlap: 51711500.0 (26.23%)
    Interval 1: [127477600.0, 179189100.0]
    Interval 2: [39275870.0, 236390800.0]
Ttest_indResult(statistic=0.6545239528346657, pvalue=0.5484996969463622)


F16BP
fold change: 0.5338737287886945
Cohen's d: 1.9570466002307463
Confidence intervals (0.95) OVERLAP
Overlap: 63566500.0 (41.76%)
    Interval 1: [105387500.0, 181279200.0]
    Interval 2: [29046050.0, 168954000.0]
Ttest_indResult(statistic=2.396882786706952, pvalue=0.0746082024667167)


Pyruvate
fold change: -0.22696515112627202
Cohen's d: -1.610322096632212
Confidence interv

<a id="session"><h3>Part 5: Print session info</h3></a>

In [1]:
!conda list

# packages in environment at C:\Users\jorda\miniconda3:
#
# Name                    Version                   Build  Channel
alabaster                 0.7.12                   pypi_0    pypi
altgraph                  0.17               pyhd8ed1ab_0    conda-forge
anyio                     3.1.0            py39hcbf5309_0    conda-forge
appdirs                   1.4.4                    pypi_0    pypi
argon2-cffi               20.1.0           py39hb82d6ee_2    conda-forge
astroid                   2.4.2            py39hde42818_1    conda-forge
async_generator           1.10                       py_0    conda-forge
attrs                     21.2.0             pyhd8ed1ab_0    conda-forge
autograd                  1.3                        py_0    conda-forge
autopep8                  1.5.4                    pypi_0    pypi
babel                     2.9.1              pyh44b312d_0    conda-forge
backcall                  0.2.0              pyh9f0ad1d_0    conda-forge
backports           